In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/research/

/content/drive/MyDrive/research


In [ ]:
%%capture
! pip install datasets transformers

In [ ]:
! pip freeze


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
import json

# New Section

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ghosh-r/bangla-gpt2")
#tokenizer = AutoTokenizer.from_pretrained("flax-community/gpt2-bengali")
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("ghosh-r/bangla-gpt2"
,pad_token_id=tokenizer.eos_token_id)

train_path = 'merged.txt'
test_path = 'test.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
sentence = 'চুপ'
numeric_ids = tokenizer.encode(sentence, return_tensors='pt')

In [ ]:
numeric_ids

tensor([[305, 280, 276]])

In [ ]:
tokenizer.decode(tokenizer.eos_token_id)

'<|endoftext|>'

In [ ]:
fid = open (test_path,'r')
count = 0
for line in fid: # treating fid as a sequence of strings
    count = count + 1
    print(line)
    
print(f"My file has {count} lines.")    
fid.close()

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("ghosh-r/bangla-gpt2")
#model = AutoModelWithLMHead.from_pretrained("flax-community/gpt2-bengali")




training_args = TrainingArguments(
    output_dir="./robi-kobita",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_steps = 1000,
    save_steps=5000, 
    warmup_steps=100,
    logging_steps=500,
    prediction_loss_only=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./robi-kobita",
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_steps = 80,
    save_steps=5000, 
    warmup_steps=100,
    logging_steps=500,
    prediction_loss_only=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "ghosh-r/bangla-gpt2",
    #"gpt2",
    vocab_size=len(tokenizer),
   
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    #num_hidden_layers=32
)
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f} M parameters")


Model size: 124.4 M parameters


In [ ]:
import matplotlib.pyplot as plt
plt.figure(1)
plt.plot(global_step)
plt.plot(train_loss)
plt.title('Step vs Loss')
plt.ylabel('Training Loss')
plt.xlabel('Step')
plt.legend(['step', 'loss'], loc='upper left')

NameError: ignored

<Figure size 640x480 with 0 Axes>

In [ ]:
trainer.train()

Step,Training Loss
500,1.634800
1000,1.506100
1500,1.433800
2000,1.374300
2500,1.323300
3000,1.282500
3500,1.243000
4000,1.209300
4500,1.188800
5000,1.170500


TrainOutput(global_step=5330, training_loss=1.3257306469314318, metrics={'train_runtime': 2773.3283, 'train_samples_per_second': 30.71, 'train_steps_per_second': 1.922, 'total_flos': 5563560591360000.0, 'train_loss': 1.3257306469314318, 'epoch': 10.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 2.2643911838531494,
 'eval_runtime': 10.2262,
 'eval_samples_per_second': 99.94,
 'eval_steps_per_second': 3.129,
 'epoch': 10.0}

In [ ]:
trainer.evaluate()


{'eval_loss': 6.268988609313965,
 'eval_runtime': 0.1748,
 'eval_samples_per_second': 91.557,
 'eval_steps_per_second': 5.722,
 'epoch': 100.0}

In [ ]:
trainer.save_model()

In [ ]:
tokenizer.save_pretrained('./tokenizer')

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/vocab.json',
 './tokenizer/merges.txt',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

#poet = pipeline('text-generation', model='ghosh-r/robi-kobi', tokenizer='ghosh-r/bangla-gpt2')
poet = pipeline('text-generation', model='/content/drive/MyDrive/research/robi-kobita', tokenizer='/content/drive/MyDrive/research/tokenizer')

In [ ]:
poem = poet('ইতিহাস এক নতুন দিকে মোড় নেয়, ', max_length=250)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
poem = poet('ওরা বসল। মিসির আলি সাহেব একটা সিগারেট ধরালেন। ', max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
ওরা বসল। মিসির আলি সাহেব একটা সিগারেট ধরালেন। নিচু গলায় বললেন, ‘সব মানুষের মধ্যেই ইএসপি কিছু পরিমাণে থাকে। টেলিপ্যাথির কথাই ধর। তোমাদের নিজেদেরই হয়তো এ বিষয়ে অভিজ্ঞতা আছে। সহজ উদাহরণ হচ্ছে, ধর, এক দিন তোমাদের কারো মনে হলো অমুকের সাথে দেখা হবে। যার সঙ্গে দেখা হবার কথা মনে হচ্ছে, সে কিন্তু এখানে থাকে না। থাকে চিটাগাং। কিন্তু সত্যি-সত্যি দেখা হয়ে গেল। কি, হয় না এ রকম?’

In [ ]:
poem = poet('কিন্তু সত্যি-সত্যি দেখা হয়ে গেল। ', max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
poem = poet('অল্প বয়সে তাহসানকে বিয়ে করা ছিল আমার জীবনের সবচেয়ে বড় ভুল, ', max_length=250)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
poem[0]['generated_text'].replace('\n', ' ').replace('\u200c', '').replace('\xa0', '')

'ওরা বসল। মিসির আলি সাহেব একটা সিগারেট ধরালেন।  ক্ৰমে চোখটা সয়ে এল। চোখ বন্ধ করে প্রত্যেকে চারটে করে সিগারেট খেল। একটা হালকা মোটা হাসি হেসে বিড়ি শেষ হল। প্রতুলবাবু সবে ফেরার পথ ধরলেন। অনুকুল ঠাকুরের দুপুরের ডিম পাড়া দেখে বললেন, আপনার বিড়ির গন্ধটাকে উপভোগ না করে বুদ্ধি খাটালাম না। অম্বরবাবু বললেন, তেওয়ারির আমলে বিড়ির একটা গন্ধের গন্ধ পেয়েছিলুম, কিন্তু এখনও পাচ'

In [ ]:
poem[0]['generated_text'].replace('\n', ' ').replace('\u200c', '').replace('\xa0', '')

'অল্প বয়সে তাহসানকে বিয়ে করা ছিল আমার জীবনের সবচেয়ে বড় ভুল,  বিশেষ করে তাহসান যখন বিবাহিত বয়সে যমজ দুটো সন্তানের পিতা। সমপত্নী? তাহসান কানে আঙুল দেখাল, না, তবে সেটা অবিশ্যি। হ্যাঁ বাবা, অনুমান করেছিলাম। বাবা হেসে বললেন, আপনার ঘরে কি ডোর টু ডোর সব জানেন আপনি? মানে, সব ছেলে? তারিখ ধরেই তাই হল। আমি ঘরে ছিলাম না, তুমি এসেছিলে। আসইনি কেন? হুঁ…আমি অনুমান করলাম, হয়ত আপনি বিশেষ ক'

In [ ]:
poem[0]['generated_text'].replace('\n', ' ').replace('\u200c', '').replace('\xa0', '')

'পরদিন সকালে উঠেই সোজা কলকাতায় চলে গেলাম। চাকর সাহেবের চাকরি নেই, তাই সাধনকে মুশকিলে পড়তে হল। এক’দিন ফেলুদার পাওনা একশো টাকার নোট তেরো হাজার, আর সেই টাকায় সাধন ট্রেনে উঠে ব্যাগ টু ব্যাগ মিষ্টি দিয়ে আপিসে যান। বাড়িতে ফিরে রাত দুটোর সময় আমরা সাধন কাকার বাড়িতে গিয়ে হাজির হই। তাঁর চাই টেলিফোনি ছিল, আর তার পরেই দেখলাম আমাদের আপিসের পিছন দিকে একটা সাধারণ ঘরে'

In [ ]:
string = str(poem[0]['generated_text'].replace('\n', ' ').replace('\xa0', '')).replace(', ', ',\n')

In [ ]:
import os

# Get a list of all the text files in the current directory
text_files = [f for f in os.listdir('.') if f.endswith('.txt')]

# Open the output file
with open('merged.txt', 'w') as outfile:
    # Loop through the list of text files
    for fname in text_files:
        # Open each file and read its contents
        with open(fname) as infile:
            contents = infile.read()
            # Write the contents to the output file
            outfile.write(contents)
